In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
HAM_PATH = '/content/drive/MyDrive/spam_classfier'
SPAM_PATH = '/content/drive/MyDrive/spam_classifier'

In [47]:
import os

HAM_DIR = os.path.join(HAM_PATH, 'easy_ham')
SPAM_DIR = os.path.join(SPAM_PATH, 'spam')

ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

In [48]:
len(ham_filenames)

2551

In [49]:
len(spam_filenames)

501

Now, this time we want to see some sample email.we want to see the first view of ham/spam email body.For this we need to load the email from the directory. we can use PYTHON EMAIL PARSER to do that task

In [50]:
import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = 'spam' if is_spam else 'easy_ham'
    with open(os.path.join(spam_path, directory, filename), 'rb') as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)


In [51]:
ham_emails = [load_email(is_spam=False,filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]


In [52]:
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [53]:
print(spam_emails[1].get_content().strip())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<HTML><HEAD>
<META content="text/html; charset=windows-1252" http-equiv=Content-Type>
<META content="MSHTML 5.00.2314.1000" name=GENERATOR></HEAD>
<BODY><!-- Inserted by Calypso -->
<TABLE border=0 cellPadding=0 cellSpacing=2 id=_CalyPrintHeader_ rules=none 
style="COLOR: black; DISPLAY: none" width="100%">
  <TBODY>
  <TR>
    <TD colSpan=3>
      <HR color=black noShade SIZE=1>
    </TD></TR></TD></TR>
  <TR>
    <TD colSpan=3>
      <HR color=black noShade SIZE=1>
    </TD></TR></TBODY></TABLE><!-- End Calypso --><!-- Inserted by Calypso --><FONT 
color=#000000 face=VERDANA,ARIAL,HELVETICA size=-2><BR></FONT></TD></TR></TABLE><!-- End Calypso --><FONT color=#ff0000 
face="Copperplate Gothic Bold" size=5 PTSIZE="10">
<CENTER>Save up to 70% on Life Insurance.</CENTER></FONT><FONT color=#ff0000 
face="Copperplate Gothic Bold" size=5 PTSIZE="10">
<CENTER>Why Spend More Than You Have To?
<CENTER><FONT color=#ff0000 face="Copp

Some emails are actually multipart with images and attachments. Let's look at the various types of structures we have:

In [54]:
def get_email_structure(email):
    #email is text/plain
    if isinstance(email, str):
        return email
    #email is payload
    payload = email.get_payload()
    if isinstance(payload, list):
        return 'multipart({})'.format(', '.join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        #then what is the content type
        return email.get_content_type()

we can use PYTHON COUNTER  for count those structure

In [55]:
from collections import Counter

def email_structure_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [56]:
email_structure_counter(ham_emails).most_common()

[('text/plain', 2453),
 ('multipart(text/plain, application/pgp-signature)', 72),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [57]:
email_structure_counter(spam_emails).most_common()

[('text/plain', 222),
 ('text/html', 181),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 19),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

It seems that the ham emails are more often plain text, while spam has quite a lot of HTML. Moreover quite a few ham emails are signed using PGP, while no spam is. In short, it seems that the email structure is useful information to have. 

let's look at email header

In [58]:
for header, value in spam_emails[1].items():
    print(header, ': ', value)

Return-Path :  <12a1mailbot1@web.de>
Delivered-To :  zzzz@localhost.example.com
Received :  from localhost (localhost [127.0.0.1])	by phobos.labs.example.com (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received :  from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received :  from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@example.com>; Thu, 22 Aug 2002 13:09:41 +0100
From :  12a1mailbot1@web.de
Received :  from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To :  dcek1a1@netsgo.com
Subject :  Life Insurance - Why Pay More?
Date :  Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version :  1.0
Message-ID :  <0103c1042001882DD_IT7@dd_it7>
Content-Type :  text/html; charset="iso-8859-1"
Content-Transfer-Encoding :  quoted-printable


In [59]:
spam_emails[1]['subject']

'Life Insurance - Why Pay More?'

okay.. 
* we saw sample ham/spam email
* we saw ham/spam emails structure
* we saw emails header, subject

Before going to the further process we will split our dataset into training and testing set.

In [60]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [61]:
# convert HTML to plain text
import re
from html import unescape

def html_to_plain_text(html):
    #remove the <head> section
    text = re.sub('<head.*?>.?</head>', '', html, flags=re.M | re.S | re.I)
    #convert <a> tag to the word HYPERLINK
    text = re.sub('<a\s.*?>', 'HYPERLINK', text, flags=re.M | re.S | re.I)
    #gets rid all of the html tag
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    #replace multiple newlines with single newlines
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S )
    return unescape(text)

let's see if it works

In [62]:
html_spam_emails = [email for email in x_train[y_train==1] if get_email_structure(email) == 'text/html']
sample_spam_email = html_spam_emails[7]
print(sample_spam_email.get_content().strip()[:1000], '...')

<html>
<head>
</head>
<center>
<h1>
<b><font face="Arial Black"><font color="#0000FF"><font size=+2>&nbsp;
Free Personal and Business Grants</font></font></font></b></h1></center>

<p>&nbsp;
<center><table BORDER=0 CELLSPACING=0 CELLPADDING=10 WIDTH="419" BGCOLOR="#0000FF" >
<tr>
<td WIDTH="397" BGCOLOR="#FFFF00">
<center>
<h2>
<font face="Arial Narrow">" Qualify for <u>at least</u> $25,000 in free
grants money - Guaranteed! "</font></h2></center>
</td>
</tr>
</table></center>

<center>
<h3>
<font face="Arial"><font size=+0>Each day over One Million Dollars in Free
Government<br>
Grants&nbsp; is given away to people just like you for a wide<br>
variety of Business And Personal Needs</font></font></h3></center>
<font face="Verdana"><font size=-1>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Dear Grant Seeker,</font></font>
<blockquote><font face="Verdana"><font size=-1>In a moment, I'll tell you
exactly <b>HOW &amp; WHERE</b> to get Grants. This <b>MONEY</b> has to
be given away, <b>WHY</b

In [63]:
print(html_to_plain_text(sample_spam_email.get_content())[:1000], '...')


 
Free Personal and Business Grants
 
" Qualify for at least $25,000 in free
grants money - Guaranteed! "
Each day over One Million Dollars in Free
Government
Grants  is given away to people just like you for a wide
variety of Business And Personal Needs
       
Dear Grant Seeker,
In a moment, I'll tell you
exactly HOW & WHERE to get Grants. This MONEY has to
be given away, WHY not to YOU?
You may be thinking, "How
can I get some of this Free Grants Money"
Maybe you think it's impossible
to get free money?
Let me tell you it's not
impossible! It's a fact, ordinary people and businesses all across the
United States are receiving millions of dollars from these Government and
Private Foundation's everyday.
Who Can Apply?
ANYONE can apply
for a Grant from 18 years old and up!
Grants from $500.00 to $50,000.00
are possible! GRANTS don't have to be paid back,
EVER! Claim
your slice of the FREE American Pie.
This money is not a loan,
Trying to get money through a conventional bank can be ver

function that takes an email and return its content as plain text

In [64]:
def email_to_text(email):
    html = None 
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ('text/plain', 'text/html'):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        if ctype == 'text/plain':
            return content
        else:
            html = content
    if html:
         return html_to_plain_text(html)
 

In [65]:
print(email_to_text(sample_spam_email)[:1000], '...')


 
Free Personal and Business Grants
 
" Qualify for at least $25,000 in free
grants money - Guaranteed! "
Each day over One Million Dollars in Free
Government
Grants  is given away to people just like you for a wide
variety of Business And Personal Needs
       
Dear Grant Seeker,
In a moment, I'll tell you
exactly HOW & WHERE to get Grants. This MONEY has to
be given away, WHY not to YOU?
You may be thinking, "How
can I get some of this Free Grants Money"
Maybe you think it's impossible
to get free money?
Let me tell you it's not
impossible! It's a fact, ordinary people and businesses all across the
United States are receiving millions of dollars from these Government and
Private Foundation's everyday.
Who Can Apply?
ANYONE can apply
for a Grant from 18 years old and up!
Grants from $500.00 to $50,000.00
are possible! GRANTS don't have to be paid back,
EVER! Claim
your slice of the FREE American Pie.
This money is not a loan,
Trying to get money through a conventional bank can be ver

let's do stemming

In [66]:
try:
    import nltk
    
    stemmer = nltk.PorterStemmer()
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(word, '=>' , stemmer.stem(word))
except ImportError:
    print("Error: steeming requires the NLTK module")
    stemmer = None

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


Replace URLS with word URL

In [67]:
!pip install urlextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
try:
    import urlextract
    url_extractor = urlextract.URLExtract()
    print(url_extractor.find_urls("Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"))
except ImportError:
    print('Error: replacing urls requiress the urlextract module.')
    url_extractor = None

['github.com', 'https://youtu.be/7Pq-S557XQU?t=3m32s']


Convert email to wordCounter

In [69]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [70]:
some_sample_email = x_train[:3]
some_sample_email_wordcounts = EmailToWordCounterTransformer().fit_transform(some_sample_email)
some_sample_email_wordcounts

array([Counter({'it': 4, 'pay': 3, 't': 3, 'the': 2, 'you': 2, 'without': 2, 's': 2, 'i': 2, 'a': 2, 'can': 2, 'look': 2, 'at': 2, 'if': 1, 'creator': 1, 'didnt': 1, 'say': 1, 'could': 1, 'have': 1, 'theft': 1, 'so': 1, 'simpl': 1, 'hell': 1, 'that': 1, 'even': 1, 'in': 1, 'all': 1, 'major': 1, 'holi': 1, 'book': 1, 'wow': 1, 've': 1, 'got': 1, 'great': 1, 'idea': 1, 'll': 1, 'hire': 1, 'skywrit': 1, 'to': 1, 'write': 1, 'thi': 1, 'then': 1, 'lock': 1, 'up': 1, 'everybodi': 1, 'who': 1, 'and': 1, 'didn': 1, 'fail': 1, 'jesu': 1, 'is': 1, 'on': 1, 'my': 1, 'side': 1, 'url': 1}),
       Counter({'i': 8, 'number': 7, 'to': 5, 'the': 5, 'of': 4, 'com': 3, 'we': 3, 'realli': 3, 'look': 3, 'it': 3, 'that': 3, 'with': 3, 'date': 2, 'welch': 2, 'panasa': 2, 't': 2, 'but': 2, 'is': 2, 'what': 2, 'would': 2, 'time': 2, 'for': 2, 'a': 2, 'have': 2, 'm': 2, 'not': 2, 'will': 2, 'exmh': 2, 'worker': 2, 'tue': 1, 'aug': 1, 'from': 1, 'brent': 1, 'messag': 1, 'id': 1, 'numbervaanumb': 1, 'blackcomb':

we have the word counts and we need to convert them to vector.

For this we will build another tranformer whose fit method will build vocabulary and transform method will use the vocabulary to convert word counts to vectors.

In [71]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word:index+1 for index,(word,count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size+1))

In [72]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(some_sample_email_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 27 stored elements in Compressed Sparse Row format>

In [73]:
X_few_vectors.toarray()

array([[ 51,   2,   0,   2,   4,   1,   1,   3,   2,   2,   2],
       [116,   8,   7,   5,   3,   5,   3,   2,   3,   1,   2],
       [ 46,   3,   5,   0,   0,   1,   1,   0,   0,   2,   0]])

In [74]:
vocab_transformer.vocabulary_

{'a': 10,
 'at': 9,
 'i': 1,
 'it': 4,
 'look': 8,
 'number': 2,
 't': 7,
 'that': 6,
 'the': 3,
 'to': 5}

we are now ready to train out first spam classifier. Tranform the whole dataset

In [83]:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ('email_to_wordcount', EmailToWordCounterTransformer()),
    ('wordcount_to_vector', WordCounterToVectorTransformer()),
])

x_train_transformed = preprocess_pipeline.fit_transform(x_train)
x_test_transformed = preprocess_pipeline.fit_transform(x_test)

Training using multiple machine learning

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [85]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=42)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=42)

In [86]:
clfs = {
    'SVC':svc,
    'KN': knc,
    'NB': mnb,
    'DT': dtc,
    'LR': lrc,
    'RF': rfc
}

In [87]:
def train(clf, features, targets):
    clf.fit(features, targets)

def predict(clf, features):
    return (clf.predict(features))

In [90]:
pred_scores = []
for classifier_name,classifier in clfs.items():
    train(classifier, x_train_transformed, y_train)
    pred = predict(classifier, x_test_transformed)
    pred_scores.append((classifier_name, [accuracy_score(y_test, pred)]))

In [91]:
pred_scores

[('SVC', [0.8494271685761048]),
 ('KN', [0.851063829787234]),
 ('NB', [0.8445171849427169]),
 ('DT', [0.7561374795417348]),
 ('LR', [0.8461538461538461]),
 ('RF', [0.8346972176759411])]

In [94]:
# using cross validation
from sklearn.model_selection import cross_val_score

score = cross_val_score(clfs['KN'], x_train_transformed, y_train, cv=3, verbose=3)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END ................................ score: (test=0.849) total time=   0.1s
[CV] END ................................ score: (test=0.843) total time=   0.2s
[CV] END ................................ score: (test=0.841) total time=   0.1s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s finished


After cross validation accuracy is 84.9. which is not bad . 

Recall and precision score

In [97]:
from sklearn.metrics import precision_score, recall_score

clfs['KN'].fit(x_train_transformed, y_train)

y_pred = clfs['KN'].predict(x_test_transformed)

print('Precision: {:.2f}'.format(100 * precision_score(y_test, y_pred)))
print('Recall: {:.2f}'.format(100 * recall_score(y_test, y_pred)))

Precision: 66.67
Recall: 2.17
